In [ ]:
import numpy as np
import pandas as pd 
import requests, xlsxwriter, math
stocks = pd.read_csv('stocks.csv')
IEX_CLOUD_API_TOKEN = ##############
symbol='SPY'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [ ]:
my_columns = ['Symbol', 'Price','MarCap', 'Shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe = final_dataframe.append(
                                        pd.Series(['SPY', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Symbol']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [ ]:
def groups(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(groups(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [ ]:
portfolio_size = input("Size of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("Portfolio size can be int or float only")
    portfolio_size = input("Size of your portfolio:")
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Symbol'])-1):
    final_dataframe.loc[i, 'Shares to buy'] = math.floor(position_size / final_dataframe['Price'][i])

writer = pd.ExcelWriter('equal_weight_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='equal_weight_trade', index = False)

In [ ]:
background_color = '#9f9fc2'
font_color = '#786c6c'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
column_formats = { 
                    'A': ['Symbol', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Shares to buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.save()